In [23]:
import os
import xml.etree.ElementTree as ET
from collections import defaultdict
from functions import *

In [24]:
LADD_PATH= '../../ladd-and-weights/dataset/full_train_ds'
HERIDAL_PATH = '../../ladd-and-weights/dataset/3rd_party/heridal'

In [25]:
# Get yolo5 code
# !git clone git@github.com:ultralytics/yolov5.git

In [26]:
# get network settings 
# !cp -r ../networks/yolo5_settings .

In [ ]:
# Generate predictions
!rm -rf predict/exp
!python ./yolov5/detect.py --augment --weights ../../ladd-and-weights/weights/yolo5/yolo5_fullDS_native.pt --source ../../ladd-and-weights/dataset/full_train_ds/JPEGImages --imgsz 1984 --conf-thres 0.05 --iou-thres 0.01 --project predict --nosave --save-txt --save-conf
# !python ./yolov5/detect.py --augment --weights ../networks/yolov5/runs/train/exp3/weights/best.pt --source ../../ladd-and-weights/dataset/full_train_ds/JPEGImages --imgsz 1984 --conf-thres 0.05 --iou-thres 0.01 --project predict --nosave --save-txt --save-conf

In [ ]:
!rm -rf predict/exp2
!python ./yolov5/detect.py --augment --weights ../../ladd-and-weights/weights/yolo5/yolo5_fullDS_native.pt --source ../../ladd-and-weights/dataset/3rd_party/heridal/JPEGImages --imgsz 1984 --conf-thres 0.05 --iou-thres 0.01 --project predict --nosave --save-txt --save-conf
# !python ./yolov5/detect.py --augment --weights ../networks/yolov5/runs/train/exp3/weights/best.pt --source ../../ladd-and-weights/dataset/3rd_party/heridal/JPEGImages --imgsz 1984 --conf-thres 0.05 --iou-thres 0.01 --project predict --nosave --save-txt --save-conf
!mv predict/exp2/labels/* predict/exp/labels/
!rm -rf predict/exp2

In [29]:
ladd_train_img = []
ladd_test_img = []
with open(os.path.join(LADD_PATH,'ImageSets/Main/train.txt'),'r') as file:
    ladd_train_img.extend([s.strip() for s in file.readlines()])
with open(os.path.join(LADD_PATH,'ImageSets/Main/val.txt'), 'r') as file:
    ladd_test_img.extend([s.strip() for s in file.readlines()])
with open(os.path.join(LADD_PATH,'ImageSets/Main/test.txt'),'r') as file:
    ladd_test_img.extend([s.strip() for s in file.readlines()])

#deduplicate (on some chunks test == val?)
ladd_test_img = list(set(ladd_test_img))
          
          
heridal_train_img = []
heridal_test_img = []
with open(os.path.join(HERIDAL_PATH,'ImageSets/Main/train.txt'), 'r') as file:
    heridal_train_img.extend([s.strip() for s in file.readlines()])
with open(os.path.join(HERIDAL_PATH,'ImageSets/Main/test.txt'), 'r') as file:
    heridal_test_img.extend([s.strip() for s in file.readlines()])

In [30]:
def parse_voc_xml(node: ET.Element):
    voc_dict = {}
    children = list(node)
    if children:
        def_dic = defaultdict(list)
        for dc in map(parse_voc_xml, children):
            for ind, v in dc.items():
                def_dic[ind].append(v)
        if node.tag == 'annotation':
            def_dic['object'] = [def_dic['object']]
        voc_dict = {
            node.tag:
                {ind: v[0] if len(v) == 1 else v
                 for ind, v in def_dic.items()}
        }
    if node.text:
        text = node.text.strip()
        if not children:
            voc_dict[node.tag] = text
    return voc_dict

In [31]:
def convert_yolo_to_pixels(size,yolo_string):
    s=[float(s) for s in  yolo_string.strip().split() ]
    center_x=int(s[1]*size[0])
    center_y=int(s[2]*size[1])
    w=int(s[3]*size[0])
    h=int(s[4]*size[1])
    x1=int(center_x-w/2)
    x2=int(center_x+w/2)
    y1=int(center_y-h/2)
    y2=int(center_y+h/2)
    return (x1,y1,x2,y2,float(s[5]))

In [32]:
def get_targets_and_preds(img_set, ds_path):
    targets = []
    predictions = []
    missed_files = 0
    for im in img_set:
        # Get true lables
        file_name = os.path.join(ds_path,'Annotations',im+'.xml')
        if not os.path.isfile(file_name):
            missed_files+=1
            # Some annotations in heridal missing???
            continue

        description = parse_voc_xml(ET.parse(file_name).getroot())
        boxes = []
        if 'annotation' in description:
            for l in description['annotation']['object']:
                bb = l['bndbox']
                boxes.append((int(bb['xmin']), int(bb['ymin']), int(bb['xmax']), int(bb['ymax'])))
            size = (int(description['annotation']['size']['width']),int(description['annotation']['size']['height']))
        else: 
            # heridal somewhere contains no size in annotations
            size = (4000,3000)
        targets.append(boxes)
        # Get predictions
        file_name = os.path.join('./predict/exp/labels',im +'.txt')
        this_predictions = []
        if os.path.exists(file_name):
            with open (file_name) as predictions_file:
                this_predictions = [str.strip(s) for s in predictions_file.readlines()]
                this_predictions = [convert_yolo_to_pixels(size,p) for p in this_predictions]
        predictions.append(this_predictions)
    if missed_files>0:
        print('warning. ',missed_files, ' xmls are missing')
    return targets,predictions

In [33]:
def evaluate_res(
    targets,
    detections, 
    iou_threshold=0.5,
    score_threshold=0.05,
    max_detections=100
):
    """ Evaluate a given dataset using a given model.
    # Arguments
        targets List [List [tuple(4)]]
        prediction List [List [ tuple (5) ]] (with score)
        targets and predictions top level list should have same len
        iou_threshold   : The threshold used to consider when a detection is positive or negative.
        score_threshold : The score confidence threshold to use for detections.
        max_detections  : The maximum number of detections to use per image.
    """
    if (len(targets) != len(detections)):
        print ("len(targets) %i != len(predictions) %i"%(len(targets),len(detections)))
        return 0

    false_positives = np.zeros((0,))
    true_positives  = np.zeros((0,))
    scores          = np.zeros((0,))
    num_annotations = 0.0

    for i in range(len(targets)):
        num_annotations     += len(targets[i])
        detected_annotations = []

        for d in range(len(detections[i])):
            if detections[i][d][4] > score_threshold:
                scores = np.append(scores, np.array(detections[i][d][4]))
                if len(targets[i]) == 0: # no objects was there
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    continue
                    
                overlaps            = compute_overlap (np.array(detections[i][d])[np.newaxis,:4],np.array(targets[i]))
                assigned_annotation = np.argmax(overlaps, axis=1)[0]
                max_overlap         = overlaps[0, assigned_annotation]
                

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)

    # F1@IoU
    plain_recall = np.sum(true_positives)/num_annotations
    plain_precision = np.sum(true_positives) / np.maximum(np.sum(true_positives) + np.sum(false_positives), np.finfo(np.float64).eps)
    F1 = 2*plain_precision*plain_recall/(plain_precision+plain_recall)


#     # sort by score
    indices         = np.argsort(-scores)
    false_positives = false_positives[indices]
    true_positives  = true_positives[indices]

#     # compute false positives and true positives
    false_positives = np.cumsum(false_positives)
    true_positives  = np.cumsum(true_positives)
#     # compute recall and precision
    recall    = true_positives / num_annotations
    precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
    # compute average precision
    average_precision  = compute_ap(recall, precision)


    return (average_precision, plain_precision, plain_recall, F1)

In [34]:
print('mAp@0.5, Precision, Recall, F1')
print('  train LADD')
targets, preds = get_targets_and_preds(ladd_train_img, LADD_PATH)
print(evaluate_res(targets, preds))
print('  test LADD')
targets, preds = get_targets_and_preds(ladd_test_img, LADD_PATH)
print(evaluate_res(targets, preds))
print('  train HERIAL')
targets, preds = get_targets_and_preds(heridal_train_img,HERIDAL_PATH)
print(evaluate_res(targets, preds))
targets, preds = get_targets_and_preds(heridal_test_img,HERIDAL_PATH)
print('  test HERIDAL')
print(evaluate_res(targets, preds))

mAp@0.5, Precision, Recall, F1
  train LADD
(0.9223420704681259, 0.7777777777777778, 0.9282200357781754, 0.8463655826281986)
  test LADD
(0.9092969535286201, 0.615, 0.9290030211480362, 0.740072202166065)
  train HERIAL
warning.  37  xmls are missing
(0.5171285184733778, 0.25826771653543307, 0.7372060857538036, 0.38252444603929314)
  test HERIDAL
(0.48281388773319944, 0.42857142857142855, 0.6231454005934718, 0.5078597339782345)
